In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import json
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
os.chdir('../')
print(os.getcwd())

c:\Users\pehbo\projects\algo


In [7]:
# Replace with your own API key and secret
with open('secrets/secrets.json') as f:
    secrets = json.load(f)
API_KEY = secrets['KEY']
API_SECRET = secrets['SECRET']
BASE_URL = 'https://paper-api.alpaca.markets'  # For paper trading, use the paper trading URL

# Set up the Alpaca API client
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

In [8]:
symbol = 'TSLA'
start_date = '2010-01-01'
end_date = '2023-03-25'

historical_data = api.get_bars(symbol, tradeapi.rest.TimeFrame.Day, start=start_date, end=end_date).df

In [ ]:
# Function to create input-output sequence for the LSTM model
def create_sequence(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

def predict_next_closing_price(model, data, scaler, window_size):
    latest_data = data[-window_size:]
    scaled_data = scaler.transform(latest_data)
    input_data = np.reshape(scaled_data, (1, window_size, 1))
    prediction = model.predict(input_data)
    return scaler.inverse_transform(prediction)[0, 0]

In [9]:
dates = historical_data.index

# Preprocess the data
close_prices = historical_data['close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_prices = scaler.fit_transform(close_prices)




Epoch 1/50
44/44 [==============================] - 8s 89ms/step - loss: 0.0463 - val_loss: 0.0067
Epoch 2/50
44/44 [==============================] - 4s 91ms/step - loss: 0.0021 - val_loss: 0.0060
Epoch 3/50
44/44 [==============================] - 5s 115ms/step - loss: 0.0012 - val_loss: 0.0072
Epoch 4/50
44/44 [==============================] - 4s 97ms/step - loss: 9.5328e-04 - val_loss: 0.0083
Epoch 5/50
44/44 [==============================] - 5s 115ms/step - loss: 8.9298e-04 - val_loss: 0.0063
Epoch 6/50
44/44 [==============================] - 5s 103ms/step - loss: 8.8796e-04 - val_loss: 0.0039
Epoch 7/50
44/44 [==============================] - 5s 111ms/step - loss: 9.2638e-04 - val_loss: 0.0032
Epoch 8/50
44/44 [==============================] - 4s 97ms/step - loss: 7.8601e-04 - val_loss: 0.0030
Epoch 9/50
44/44 [==============================] - 5s 108ms/step - loss: 7.5465e-04 - val_loss: 0.0024
Epoch 10/50
44/44 [==============================] - 5s 107ms/step - loss: 7.379

In [ ]:
# Create the input-output sequences for the LSTM model
window_size = 60
X, y = create_sequence(scaled_close_prices, window_size)

# Split the data into training (80%) and testing (20%) sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the LSTM model
model = Sequential([
    LSTM(units=50, activation='relu', input_shape=(window_size, 1), return_sequences=True),
    LSTM(units=50, activation='relu', return_sequences=True),
    LSTM(units=50, activation='relu'),
    Dense(units=1)
])

# Compile and train the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



In [ ]:
# Predict the next closing price
next_closing_price = predict_next_closing_price(model, close_prices, scaler, window_size)
print(f'Predicted next closing price: {next_closing_price}')